In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.accelerometer import plot_fourier_transformation, plot_acceleration, plot_feature_columns, accelerometer_feature_engineering
from src.kmeans import kmeans
from src.principal_component_analysis import principal_component_analysis, plot_principal_component_analysis
from src.decision_tree import decision_tree
from src.plotting import box_plot_columns
from src.time_series import median_filter, run_time_series_algorithms
from src.ml_util import run_feature_algorithms

In [ ]:
slow_move_df = pd.read_csv('slow_move_accelerations.csv')
slow_move_df = slow_move_df[slow_move_df['age_group']!=0]

slow_move_touches_df = pd.read_csv('slow_move_touches.csv')
slow_move_touches_df = slow_move_touches_df[slow_move_touches_df['age_group']!=0]

In [ ]:
slow_move_df.head(5)

In [ ]:
slow_move_touches_df.head()

In [ ]:
slow_move_df.describe()

In [ ]:
single_session_df = slow_move_df[slow_move_df['uuid'] == slow_move_df['uuid'].unique()[0]]

In [ ]:
plot_acceleration(single_session_df, subplots=False)

In [ ]:
single_session_df = median_filter(single_session_df)
plot_acceleration(single_session_df, subplots=False)

In [ ]:
plot_fourier_transformation(single_session_df, 'test session')

In [ ]:
box_plot_columns(single_session_df)

In [ ]:
plot_acceleration(single_session_df)

In [ ]:
slow_move_df = median_filter(slow_move_df)

In [ ]:
slow_move_processed_df = accelerometer_feature_engineering(slow_move_df)
slow_move_processed_df = slow_move_processed_df.reset_index(drop=False)

In [ ]:
slow_move_touches_df['center_line_deviation'] = np.abs(slow_move_touches_df['y'] - slow_move_touches_df['start_y'])

In [ ]:
slow_move_processed_df['y_touch_std'] =  np.nan
for uuid in slow_move_touches_df['uuid'].unique():
    current_session = slow_move_touches_df[slow_move_touches_df['uuid']==uuid]
    y_touch_std = current_session['x'].std()
    slow_move_processed_df.loc[slow_move_processed_df['uuid'] == uuid, 'y_touch_std'] = y_touch_std
    slow_move_processed_df.loc[slow_move_processed_df['uuid'] == uuid, 'center_line_deviation'] = current_session['center_line_deviation'].mean()

In [ ]:
slow_move_processed_df.head()

In [ ]:
slow_move_processed_df.corr()

SEM is closely related to std -> therefore we drop it

In [ ]:
slow_move_processed_df = slow_move_processed_df.drop(columns=['x_sem', 'y_sem', 'z_sem', 'mag_sem'])

In [ ]:
plot_feature_columns(slow_move_processed_df,'std')
plot_feature_columns(slow_move_processed_df,'mean')
plot_feature_columns(slow_move_processed_df,'peaks')
plot_feature_columns(slow_move_processed_df,'snr')
plot_feature_columns(slow_move_processed_df,'sal')
box_plot_columns(slow_move_processed_df, 'age_group',['duration'])
box_plot_columns(slow_move_processed_df, 'age_group',['y_touch_std'])
box_plot_columns(slow_move_processed_df, 'age_group',['center_line_deviation'])

We can see that there is no significant deviation between the two age groups in the following features:
- y_mean
- mag_mean
- z_snr
- mag_snr
- z_sal
- mag_sal

# ML Models

In [ ]:
results = {} 

In [ ]:
feature_keys = ['y_std','z_std','x_mean','z_peaks','center_line_deviation']
class_key = ['age_group']

## Principal Component Analysis

In [ ]:
principal_components_df = principal_component_analysis(slow_move_processed_df, feature_keys)

In [ ]:
plot_principal_component_analysis(slow_move_processed_df.reset_index(drop=False), principal_components_df, 'age_group', [30,50])

## Run Algorithms on extracted Features

In [ ]:
feature_results = run_feature_algorithms(slow_move_processed_df, feature_keys)
results.update(feature_results)

## Run Time Series Algorithm

In [ ]:
time_series_results = run_time_series_algorithms(slow_move_df)
results.update(time_series_results)

## ML Results

In [ ]:
results